# RPA trazer informações do banco do 2o para o banco do 1o

In [2]:
import os
import psycopg2
# from pymongo import MongoClient
import pandas as pd
from dotenv import load_dotenv
# import pymongo
# from sqlalchemy import create_engine

C:\Users\oliviadomingues-ieg\AppData\Local\Temp\ipykernel_13364\4197665263.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
load_dotenv()

host_1ano = os.getenv("DB_HOST_1ano")
port_1ano = int(os.getenv("DB_PORT_1ano"))
user_1ano = os.getenv("DB_USER_1ano")
password_1ano = os.getenv("DB_PASSWORD_1ano")
database_1ano = os.getenv("DB_NAME_1ano")
db_url_1ano = os.getenv("DATABASE_URL_1ano")

host_2ano = os.getenv("DB_HOST_2ano")
port_2ano = int(os.getenv("DB_PORT_2ano"))
user_2ano = os.getenv("DB_USER_2ano")
password_2ano = os.getenv("DB_PASSWORD_2ano")
database_2ano = os.getenv("DB_NAME_2ano")
db_url_2ano = os.getenv("DATABASE_URL_2ano")

print(database_2ano)
print(database_1ano)
# conexao = MongoClient(os.getenv("DB_MONGO_URL"))
# db_name = conexao[os.getenv("DB_MONGO_NAME")]
# collection_funcoes = db_name[os.getenv("DB_MONGO_COLLECTION_FUNCOES")] 
# collection_situacao_trabalhista = db_name[os.getenv("DB_MONGO_COLLECTION_SITUACAO_TRABALHISTA")]

db_incluses_2o
db_incluses_1o


In [46]:
try:
    # conectando-se ao banco do 1ano
    conn_bd1 = psycopg2.connect(
        user=user_1ano,
        host=host_1ano,
        port=port_1ano,
        password=password_1ano,
        database=database_1ano,
    )

    # conectando-se ao banco do 2ano
    conn_bd2 = psycopg2.connect(
        user=user_2ano,
        host=host_2ano,
        port=port_2ano,
        password=password_2ano,
        database=database_2ano,
    )
    
except Exception as e:
    print("Error connecting to PostgreSQL:", e)

In [10]:
# try:
#     # conectando-se ao banco do 1ano
#     conn_bd1 = create_engine(db_url_1ano)

#     # conectando-se ao banco do 2ano
#     conn_bd2 = create_engine(db_url_2ano)
    
# except Exception as e:
#     print("Error connecting to PostgreSQL:", e)

In [47]:
# Criando um cursores para fazer queries em ambos os bancos
cursor_bd1 = conn_bd1.cursor()
cursor_bd2 = conn_bd2.cursor()

## tipo_vaga

In [17]:
# Lendo os valores da tabela tipo_vaga do 2ano
df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM tipo_vaga;", conn_bd2)

try:
    for i in range(len(df_tipo_vaga_bd2)):
        # Lendo os valores de cada linha do dataframe que contém as informações da tabela tipo_vaga do 2ano
        values2o = (df_tipo_vaga_bd2['nome'][i], df_tipo_vaga_bd2['id'][i])
        
        # Query para atualizar a tabela do 1o com as informações do 2o
        update_query = "UPDATE public.tipo_vaga SET nome = %s WHERE id = %s;"

        # Executando a query de atualização
        cursor_bd1.execute(update_query, values2o)
        
        # Se a query de atualização não tiver nenhuma linha alterada, então significa que os valores na tabela do 2o são novos
        if cursor_bd1.rowcount == 0:
            # query para inserir
            insert_query = "INSERT INTO public.tipo_vaga(nome, id) VALUES (%s, %s);"

            # Executando a query de insert
            cursor_bd1.execute(insert_query, values2o)

    conn_bd1.commit()  # Commit das alterações no bd1

except Exception as e:
    print(f"Ocorreu um erro: {e}")


C:\Users\oliviadomingues-ieg\AppData\Local\Temp\ipykernel_13364\2540846623.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM tipo_vaga;", conn_bd2)


## permissao_vaga

In [26]:
# Lendo os valores da tabela tipo_vaga do 2ano
df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM permissao_vaga;", conn_bd2)

try:
    for i in range(len(df_tipo_vaga_bd2)):

        # Lendo os valores de cada linha do dataframe que contém as informações da tabela tipo_vaga do 2ano
        # isso pode ser a versão do meu driver
        values2o = (bool(df_tipo_vaga_bd2['permissao'][i]), df_tipo_vaga_bd2['id'][i], df_tipo_vaga_bd2['fk_vaga_id'][i])

        # Query para atualizar a tabela do 1o com as informações do 2o
        update_query = "UPDATE public.permissao_vaga SET permissao = %s WHERE id = %s;"

        # Executando a query de atualização
        cursor_bd1.execute(update_query, (values2o[0], values2o[1]))

        # Se a query de atualização não tiver nenhuma linha alterada, então significa que os valores na tabela do 2o são novos
        if cursor_bd1.rowcount == 0:
            # query para inserir
            insert_query = "INSERT INTO public.permissao_vaga(permissao, id, fk_vaga_id) VALUES (%s, %s, %s);"

            # Executando a query de insert
            cursor_bd1.execute(insert_query, values2o)

    # Queries para pegar os IDs da 
    cursor_bd2.execute("SELECT id FROM permissao_vaga;")
    ids_bd2 = [id_[0] for id_ in cursor_bd2.fetchall()]

    # Deletando todos os registros na tabela do 1o que não estão presentes na tabela do 2o
    delete_query = f"""
        DELETE FROM public.permissao_vaga
        WHERE id NOT IN {tuple(ids_bd2)}
    """

    cursor_bd1.execute(delete_query)

    conn_bd1.commit()  # Commit das alterações no bd1

except Exception as e:
    print(f"Ocorreu um erro: {e}")

C:\Users\oliviadomingues-ieg\AppData\Local\Temp\ipykernel_13364\2017490817.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM permissao_vaga;", conn_bd2)


## situacao_trabalhista

In [27]:
# Lendo os valores da tabela tipo_vaga do 2ano
df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM situacao_trabalhista;", conn_bd2)

try:
    for i in range(len(df_tipo_vaga_bd2)):
        # Lendo os valores de cada linha do dataframe que contém as informações da tabela tipo_vaga do 2ano
        values2o = (df_tipo_vaga_bd2['nome'][i], df_tipo_vaga_bd2['id'][i])
        
        # Query para atualizar a tabela do 1o com as informações do 2o
        update_query = "UPDATE public.situacao_trabalhista SET nome = %s WHERE id = %s;"

        # Executando a query de atualização
        cursor_bd1.execute(update_query, values2o)

        # Se a query de atualização não tiver nenhuma linha alterada, então significa que os valores na tabela do 2o são novos
        if cursor_bd1.rowcount == 0:
            # query para inserir
            insert_query = "INSERT INTO public.situacao_trabalhista(nome, id) VALUES (%s, %s);"

            # Executando a query de insert
            cursor_bd1.execute(insert_query, values2o)

    conn_bd1.commit()  # Commit das alterações no bd1

except Exception as e:
    print(f"Ocorreu um erro: {e}")

C:\Users\oliviadomingues-ieg\AppData\Local\Temp\ipykernel_13364\1474728197.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM situacao_trabalhista;", conn_bd2)


## setor

In [31]:
# Lendo os valores da tabela tipo_vaga do 2ano
df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM setor;", conn_bd2)

try:
    for i in range(len(df_tipo_vaga_bd2)):
        # Lendo os valores de cada linha do dataframe que contém as informações da tabela tipo_vaga do 2ano
        values2o = (df_tipo_vaga_bd2['nome'][i], df_tipo_vaga_bd2['id'][i])
        
        # Query para atualizar a tabela do 1o com as informações do 2o
        update_query = "UPDATE public.setor SET nome = %s WHERE id = %s;"

        # Executando a query de atualização
        cursor_bd1.execute(update_query, values2o)

        # Se a query de atualização não tiver nenhuma linha alterada, então significa que os valores na tabela do 2o são novos
        if cursor_bd1.rowcount == 0:
            # query para inserir
            insert_query = "INSERT INTO public.setor(nome, id) VALUES (%s, %s);"

            # Executando a query de insert
            cursor_bd1.execute(insert_query, values2o)

    conn_bd1.commit()  # Commit das alterações no bd1
    #conn_bd2.commit()  # Commit das alterações no bd2

except Exception as e:
    print(f"Ocorreu um erro: {e}")


C:\Users\oliviadomingues-ieg\AppData\Local\Temp\ipykernel_13364\671103324.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM setor;", conn_bd2)


## tipo_arquivo

In [29]:
# Lendo os valores da tabela tipo_vaga do 2ano
df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM tipo_arquivo;", conn_bd2)

try:
    for i in range(len(df_tipo_vaga_bd2)):
        # Lendo os valores de cada linha do dataframe que contém as informações da tabela tipo_vaga do 2ano
        values2o = (df_tipo_vaga_bd2['nome'][i], df_tipo_vaga_bd2['id'][i])

        # Query para atualizar a tabela do 1o com as informações do 2o
        update_query = "UPDATE public.tipo_arquivo SET nome = %s WHERE id = %s;"

        # Executando a query de atualização
        cursor_bd1.execute(update_query, values2o)

        # Se a query de atualização não tiver nenhuma linha alterada, então significa que os valores na tabela do 2o são novos
        if cursor_bd1.rowcount == 0:
            # query para inserir
            insert_query = "INSERT INTO public.tipo_arquivo(nome, id) VALUES (%s, %s);"

            # Executando a query de insert
            cursor_bd1.execute(insert_query, values2o)
            
        conn_bd1.commit()  # Commit das alterações no bd1

except Exception as e:
    print(f"Ocorreu um erro ao processar o registro {i}: {e}")


('PDF', 'e7a9de83-6cc5-4db8-a6c1-899468571d85')
Rowcount:  1
('Imagem', 'f4776511-213e-4a63-abf2-1de797615d47')
Rowcount:  1
('Documento', '715e717b-04d5-4b1d-9c3a-21e4f336d801')
Rowcount:  1
('DOC', '840fe6bb-49dd-4e4f-9779-f5ccf783d87c')
Rowcount:  1
('DOCX', 'e789d47c-8c96-4a28-aa7c-c0a5743d51a0')
Rowcount:  1
('Texto', '1823cbac-a018-4cce-a642-d3a8b1c1f0d0')
Rowcount:  1
('Planilha', '2e56845d-84ff-47e4-a3f9-64c6ae4f84ad')
Rowcount:  1
('Apresentação', 'cf97e830-1324-47c6-a85c-eb5ac35765b7')
Rowcount:  1


C:\Users\oliviadomingues-ieg\AppData\Local\Temp\ipykernel_13364\48969208.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tipo_vaga_bd2 = pd.read_sql_query("SELECT * FROM tipo_arquivo;", conn_bd2)


## permissao_curso

In [48]:
# Lendo os valores da tabela permissao_curso do 2ano
df_permissao_curso = pd.read_sql_query("SELECT * FROM permissao_curso;", conn_bd2)

try:
    for i in range(len(df_permissao_curso)):
        # Lendo os valores de cada linha do dataframe
        values2o = (bool(df_permissao_curso['permissao'][i]), df_permissao_curso['id'][i], df_permissao_curso['fk_curso_id'][i])

        # Query para atualizar a tabela do 1o com as informações do 2o
        update_query = "UPDATE public.permissao_curso SET permissao = %s WHERE id = %s;"
        cursor_bd1.execute(update_query, (values2o[0], values2o[1]))

        # Se a query de atualização não tiver nenhuma linha alterada, insira os novos valores
        if cursor_bd1.rowcount == 0:
            insert_query = "INSERT INTO public.permissao_curso (permissao, id, fk_curso_id) VALUES (%s, %s, %s);"
            cursor_bd1.execute(insert_query, values2o)

    # Pegando os IDs da tabela permissao_vaga no 2o banco
    cursor_bd2.execute("SELECT id FROM permissao_vaga;")
    ids_bd2 = [id_[0] for id_ in cursor_bd2.fetchall()]

    # Verificando a quantidade de IDs para deletar registros
    if ids_bd2:
        # Deletando registros do 1o banco que não estão no 2o
        delete_query = f"DELETE FROM public.permissao_vaga WHERE id NOT IN {tuple(ids_bd2)}"
        cursor_bd1.execute(delete_query)

    # Commit das alterações no bd1
    conn_bd1.commit()

except Exception as e:
    print(f"Ocorreu um erro: {e}")


                                     id  permissao  \
0  d0b8ee46-4191-4886-8c57-c0fb7cacac7c       True   
1  32af9acd-6120-4be3-8aa9-134f8cf91206      False   

                            fk_curso_id  
0  cf9ffd85-d564-46c5-9a07-7cd42393bda5  
1  6245647a-c84b-4c0c-9239-dac8d2eaac1b  


C:\Users\oliviadomingues-ieg\AppData\Local\Temp\ipykernel_13364\4083892168.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_permissao_curso = pd.read_sql_query("SELECT * FROM permissao_curso;", conn_bd2)


In [19]:
conn_bd2.close()
conn_bd1.close()
